Corpus deduplication via binning

In [1]:
import pandas as pd
import py_stringmatching as sm

Import in the data

In [17]:
corpus_folder = '../data/corpi/'
corpus_file = 'neighborhood_corpus.csv'
corpus_header = 'neighborhood'

In [8]:
corpus = pd.read_csv(corpus_folder + corpus_file)

Initialize the string similarity metric

In [10]:
jaro = sm.Jaro()
print(jaro.get_sim_score('the', 'theft'))

0.8666666746139526


Do cartesian product of corpus for comparisons

In [15]:
corpus_cross = corpus.merge(corpus, how='cross')
corpus_cross

,neighborhood_x,neighborhood_y
0,clairemont mesa east,clairemont mesa east
1,clairemont mesa east,clairemont mesa west
2,clairemont mesa east,bay ho
3,clairemont mesa east,north clairemont
4,clairemont mesa east,university city
...,...,...
161599,via de la valle,rolando village
161600,via de la valle,san pasqual valley
161601,via de la valle,sorrento mesa
161602,via de la valle,torrey hills


In [19]:
def get_jaro_sim_score(x):
    return jaro.get_sim_score(x[corpus_header + '_x'], x[corpus_header + '_y'])

In [20]:
corpus_cross['jaro_score'] = corpus_cross.apply(get_jaro_sim_score, axis=1)

In [21]:
corpus_cross

,neighborhood_x,neighborhood_y,jaro_score
0,clairemont mesa east,clairemont mesa east,1.000000
1,clairemont mesa east,clairemont mesa west,0.966667
2,clairemont mesa east,bay ho,0.438889
3,clairemont mesa east,north clairemont,0.644444
4,clairemont mesa east,university city,0.519444
...,...,...,...
161599,via de la valle,rolando village,0.622222
161600,via de la valle,san pasqual valley,0.674074
161601,via de la valle,sorrento mesa,0.358120
161602,via de la valle,torrey hills,0.450000
